In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

In [2]:
df = pd.read_csv('1980.csv')

In [3]:
df['genres'].unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'bluegrass', 'blues', 'brazil', 'british', 'cantopop',
       'chicago-house', 'children', 'chill', 'classical', 'comedy',
       'country', 'dance', 'dancehall', 'deep-house', 'disco', 'dub',
       'dubstep', 'edm', 'electro', 'electronic', 'folk', 'forro',
       'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'indian', 'indie',
       'indie-pop', 'industrial', 'j-idol', 'j-pop', 'j-rock', 'jazz',
       'k-pop', 'latin', 'latino', 'malay', 'mandopop', 'metal',
       'minimal-techno', 'mpb', 'new-age', 'opera', 'pagode', 'party',
       'piano', 'pop', 'pop-film', 'power-pop', 'progressive-house',
       'psych-rock', 'punk', 'punk-rock', 'r-n-b', 'reggae', 'reggaeton',
       'rock', 'rock-n-roll', 'rockabilly', 'salsa', 'samba', 'sertanejo',
       '

Check for duplicate values

In [4]:
df.shape

(293997, 24)

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.shape

(76414, 24)

There were a lot of duplicate values..

Check for null values

In [7]:
df.isna().any()

name                      False
artists                   False
album                     False
release_date              False
release_date_precision    False
chart_power                True
spotify_id                False
uri                       False
popularity                False
danceability              False
energy                    False
key                       False
loudness                  False
mode                      False
speechiness               False
acousticness              False
instrumentalness          False
liveness                  False
valence                   False
tempo                     False
duration_ms               False
time_signature            False
isrc                       True
genres                    False
dtype: bool

There exist Null values for the Chart Power and for the isrc. The null values for the Chart Power are valid, because not every song was in the Charts. Lets have a look at the null values for isrc:

In [8]:
df[df['isrc'].isna()]

,name,artists,album,release_date,release_date_precision,chart_power,spotify_id,uri,popularity,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,isrc,genres
171526,Amor de Verano,Pintura Roja,Amor de Verano,1980,year,NaN,2ndwh9yRCFK3CA5zhaJWXY,spotify:track:2ndwh9yRCFK3CA5zhaJWXY,32,0.515,...,0.0352,0.00685,0.706,0.0737,0.909,105.044,204202,4,NaN,pop


Select all categorical features and group by the spotify id. All values of the features are stored as lists.

In [9]:
df_categorical = df[['spotify_id', 'genres', 'name', 'artists', 'album', 'release_date', 'release_date_precision', 'uri', 'isrc']]

In [10]:
display(df_categorical.head())
df_categorical.shape

,spotify_id,genres,name,artists,album,release_date,release_date_precision,uri,isrc
0,5xgPirJuHEYxM3pMFPYtXG,acoustic,My Baby Has Eyes Like An Eagle - Take 2,Mississippi Fred McDowell,Levee Camp Blues,1980-01-01,day,spotify:track:5xgPirJuHEYxM3pMFPYtXG,US2AH0401019
1,6vonkFQZRt9WHHfO6iU1AL,acoustic,Round And Round,J.B. Lenoir,Down in Mississippi,1980,year,spotify:track:6vonkFQZRt9WHHfO6iU1AL,DER387900019
2,7pFvU6cr3CURlNc8S9SxAp,acoustic,Ain't Doin' No Good,Johnny Shines,Johnny Shines and Robert Lockwood,1980-01-14,day,spotify:track:7pFvU6cr3CURlNc8S9SxAp,US2Y30629315
3,6CR1NaaMBPQUed2i5QRW4l,acoustic,Gonna Call the Angel,Johnny Shines,Johnny Shines and Robert Lockwood,1980-01-14,day,spotify:track:6CR1NaaMBPQUed2i5QRW4l,US2Y30629323
4,1td7cg3olnhudXRKSASjRZ,acoustic,Gonna Call the Angel - Master Take,Johnny Shines,Johnny Shines and Robert Lockwood,1980-01-14,day,spotify:track:1td7cg3olnhudXRKSASjRZ,US2Y30629321


(76414, 9)

In [11]:
df_categorical = df_categorical.groupby('spotify_id').agg(lambda x: list(x)).reset_index()

In [12]:
display(df_categorical.head())
df_categorical.shape

,spotify_id,genres,name,artists,album,release_date,release_date_precision,uri,isrc
0,000P83HDtOHcNVFZy7Q2Yu,[salsa],[El Licor de Tu Boquita],[El Gran Combo De Puerto Rico],[Unity],[1980],[year],[spotify:track:000P83HDtOHcNVFZy7Q2Yu],[USMRE0801806]
1,000paUq6xoYcvCz2REJdu1,"[pop, pop-film]","[Hun Tey Kharkan Gay Alarum, Hun Tey Kharkan G...","[Noor Jehan, Noor Jehan]","[Noor Jehan New Punjabi Songs, Noor Jehan New ...","[1980-12-01, 1980-12-01]","[day, day]","[spotify:track:000paUq6xoYcvCz2REJdu1, spotify...","[USA561127553, USA561127553]"
2,0017A9myIXhXYB4gBsPRsA,[pop],[Untuk Sebuah Nama],[Ratih Purwasih],[Untuk Sebuah Nama],[1980],[year],[spotify:track:0017A9myIXhXYB4gBsPRsA],[UKFT41826340]
3,001reycpvPkUjmpxTJv6eV,"[classical, german]","[Oboe Concerto in F Minor, TWV 51:f1: I. Alleg...","[Georg Philipp Telemann,Burkhard Glaetzner,Har...",[Oboe Concertos (Baroque) - Antonio Vivaldi: G...,"[1980, 1980]","[year, year]","[spotify:track:001reycpvPkUjmpxTJv6eV, spotify...","[DES380744015, DES380744015]"
4,002H0J6N0wKjjRtFQhKDzZ,"[dance, turkish]","[Arabesk Oyun Havası, Arabesk Oyun Havası]","[Esin Engin Orkestrası, Esin Engin Orkestrası]","[Arabesk'80, Arabesk'80]","[1980, 1980]","[year, year]","[spotify:track:002H0J6N0wKjjRtFQhKDzZ, spotify...","[BGA472127386, BGA472127386]"


(38456, 9)

Select all numerical features and group them by the spotify id.

In [13]:
df_numerical = df.drop(columns=['genres', 'name', 'artists', 'album', 'release_date', 'release_date_precision', 'uri', 'isrc'])

In [14]:
df_numerical.drop_duplicates(inplace=True)

In [15]:
df_numerical.head()

,chart_power,spotify_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,NaN,5xgPirJuHEYxM3pMFPYtXG,8,0.600,0.118,3,-18.452,1,0.0402,0.977,0.072900,0.1100,0.557,98.429,278787,4
1,NaN,6vonkFQZRt9WHHfO6iU1AL,5,0.887,0.344,1,-11.985,1,0.0481,0.832,0.028000,0.3370,0.620,129.867,171445,4
2,NaN,7pFvU6cr3CURlNc8S9SxAp,1,0.433,0.197,8,-13.150,1,0.0324,0.993,0.003270,0.1090,0.804,85.316,155840,4
3,NaN,6CR1NaaMBPQUed2i5QRW4l,1,0.442,0.167,2,-14.996,1,0.0358,0.859,0.001730,0.0876,0.243,72.212,189427,4
4,NaN,1td7cg3olnhudXRKSASjRZ,0,0.468,0.205,2,-13.606,0,0.0491,0.778,0.000237,0.0832,0.374,114.904,209107,3


In [16]:
df_numerical.shape

(39520, 16)

In [17]:
df_numerical = df_numerical.groupby('spotify_id').mean()

In [18]:
df_numerical.shape

(38456, 15)

In [19]:
df_numerical.reset_index(inplace=True)

Merge both DataFrames grouped by the spotify id together.

In [20]:
df_total = df_categorical.merge(df_numerical, how='left', on='spotify_id')

In [21]:
df_total

,spotify_id,genres,name,artists,album,release_date,release_date_precision,uri,isrc,chart_power,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,000P83HDtOHcNVFZy7Q2Yu,[salsa],[El Licor de Tu Boquita],[El Gran Combo De Puerto Rico],[Unity],[1980],[year],[spotify:track:000P83HDtOHcNVFZy7Q2Yu],[USMRE0801806],NaN,...,-8.377,0.0,0.0736,0.4380,0.000042,0.1260,0.696,88.940,242493.0,4.0
1,000paUq6xoYcvCz2REJdu1,"[pop, pop-film]","[Hun Tey Kharkan Gay Alarum, Hun Tey Kharkan G...","[Noor Jehan, Noor Jehan]","[Noor Jehan New Punjabi Songs, Noor Jehan New ...","[1980-12-01, 1980-12-01]","[day, day]","[spotify:track:000paUq6xoYcvCz2REJdu1, spotify...","[USA561127553, USA561127553]",NaN,...,-9.944,1.0,0.0868,0.7410,0.037700,0.2220,0.606,96.169,372167.0,4.0
2,0017A9myIXhXYB4gBsPRsA,[pop],[Untuk Sebuah Nama],[Ratih Purwasih],[Untuk Sebuah Nama],[1980],[year],[spotify:track:0017A9myIXhXYB4gBsPRsA],[UKFT41826340],NaN,...,-3.549,1.0,0.0289,0.0840,0.000004,0.1170,0.360,81.433,249504.0,4.0
3,001reycpvPkUjmpxTJv6eV,"[classical, german]","[Oboe Concerto in F Minor, TWV 51:f1: I. Alleg...","[Georg Philipp Telemann,Burkhard Glaetzner,Har...",[Oboe Concertos (Baroque) - Antonio Vivaldi: G...,"[1980, 1980]","[year, year]","[spotify:track:001reycpvPkUjmpxTJv6eV, spotify...","[DES380744015, DES380744015]",NaN,...,-14.759,0.0,0.0348,0.8310,0.916000,0.1290,0.540,112.185,215733.0,4.0
4,002H0J6N0wKjjRtFQhKDzZ,"[dance, turkish]","[Arabesk Oyun Havası, Arabesk Oyun Havası]","[Esin Engin Orkestrası, Esin Engin Orkestrası]","[Arabesk'80, Arabesk'80]","[1980, 1980]","[year, year]","[spotify:track:002H0J6N0wKjjRtFQhKDzZ, spotify...","[BGA472127386, BGA472127386]",NaN,...,-12.444,0.0,0.0365,0.5280,0.323000,0.3430,0.752,134.274,345943.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38451,7zsH8mPq4nPBqqqDJpjgJe,[pop],[Nyalakan Api],[Deddy Dores],[Nyalakan Api],[1980],[year],[spotify:track:7zsH8mPq4nPBqqqDJpjgJe],[UKFT41809851],NaN,...,-13.378,0.0,0.0293,0.0266,0.000000,0.4220,0.257,134.941,346514.0,4.0
38452,7zv3sIP0Umtz9MswoJSF1T,[country],[O Little Town of Bethlehem],[Johnny Cash],[Classic Christmas],[1980-12-01],[day],[spotify:track:7zv3sIP0Umtz9MswoJSF1T],[USSM10310503],NaN,...,-17.087,0.0,0.0314,0.9400,0.000000,0.1370,0.187,86.192,206760.0,4.0
38453,7zwHfIlTAIPa6hgjzK1MGx,[folk],"[Mac's fancy, The Mist Covered Mountain]",[De Dannan],[The Mist Covered Mountain],[1980-01-01],[day],[spotify:track:7zwHfIlTAIPa6hgjzK1MGx],[USA560555553],NaN,...,-13.629,1.0,0.0525,0.7580,0.960000,0.1800,0.942,126.660,188546.0,4.0
38454,7zwlOlX3MA7BFkmf0c1y4p,"[guitar, jazz]","[Bianca's Midnight Lullaby, Bianca's Midnight ...","[Al Di Meola, Al Di Meola]","[Splendido Hotel, Splendido Hotel]","[1980, 1980]","[year, year]","[spotify:track:7zwlOlX3MA7BFkmf0c1y4p, spotify...","[USSM10008513, USSM10008513]",NaN,...,-18.132,1.0,0.0439,0.9070,0.866000,0.1100,0.452,78.743,113827.0,3.0


Create a DataFrame which contains only the relevant features for the recommender system

In [22]:
df_recommender = df.drop(columns=['artists', 'genres','album', 'release_date', 'release_date_precision', 'chart_power', 'uri', 'popularity', 'name', 'isrc'])

Drop duplicate values..

In [23]:
df_recommender.shape

(76414, 14)

In [24]:
df_recommender.drop_duplicates(inplace=True)

In [25]:
df_recommender.shape

(38456, 14)

There were a lot of duplicates..

In [26]:
df_recommender.head()

,spotify_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5xgPirJuHEYxM3pMFPYtXG,0.600,0.118,3,-18.452,1,0.0402,0.977,0.072900,0.1100,0.557,98.429,278787,4
1,6vonkFQZRt9WHHfO6iU1AL,0.887,0.344,1,-11.985,1,0.0481,0.832,0.028000,0.3370,0.620,129.867,171445,4
2,7pFvU6cr3CURlNc8S9SxAp,0.433,0.197,8,-13.150,1,0.0324,0.993,0.003270,0.1090,0.804,85.316,155840,4
3,6CR1NaaMBPQUed2i5QRW4l,0.442,0.167,2,-14.996,1,0.0358,0.859,0.001730,0.0876,0.243,72.212,189427,4
4,1td7cg3olnhudXRKSASjRZ,0.468,0.205,2,-13.606,0,0.0491,0.778,0.000237,0.0832,0.374,114.904,209107,3


Since all features except the spotify_id are numerical features and the spotify_id should be an unique identifier, try to group by the id to reduce the DataFrame even more.

In [27]:
df_recommender = df_recommender.groupby(['spotify_id']).mean()

In [28]:
display(df_recommender.head())
df_recommender.shape

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,0.619,0.561,7.0,-8.377,0.0,0.0736,0.438,0.000042,0.126,0.696,88.940,242493.0,4.0
000paUq6xoYcvCz2REJdu1,0.492,0.558,2.0,-9.944,1.0,0.0868,0.741,0.037700,0.222,0.606,96.169,372167.0,4.0
0017A9myIXhXYB4gBsPRsA,0.220,0.668,0.0,-3.549,1.0,0.0289,0.084,0.000004,0.117,0.360,81.433,249504.0,4.0
001reycpvPkUjmpxTJv6eV,0.448,0.224,5.0,-14.759,0.0,0.0348,0.831,0.916000,0.129,0.540,112.185,215733.0,4.0
002H0J6N0wKjjRtFQhKDzZ,0.286,0.480,2.0,-12.444,0.0,0.0365,0.528,0.323000,0.343,0.752,134.274,345943.0,4.0


(38456, 13)

The shape stayed the same, so there were no duplicate ids.

Scale the data, so every feature has the same influence.

In [29]:
scaler = MinMaxScaler()
df_recommender_scaled = scaler.fit_transform(df_recommender)
df_recommender_scaled = pd.DataFrame(df_recommender_scaled, columns=df_recommender.columns, index = df_recommender.index)

In [30]:
df_recommender_scaled.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,0.638803,0.561519,0.636364,0.798337,0.0,0.076507,0.439758,0.000042,0.114847,0.696,0.387169,0.067168,0.8
000paUq6xoYcvCz2REJdu1,0.507740,0.558516,0.181818,0.764918,1.0,0.090229,0.743976,0.037851,0.212072,0.606,0.418638,0.104717,0.8
0017A9myIXhXYB4gBsPRsA,0.227038,0.668637,0.000000,0.901301,1.0,0.030042,0.084336,0.000004,0.105732,0.360,0.354490,0.069198,0.8
001reycpvPkUjmpxTJv6eV,0.462332,0.224149,0.454545,0.662231,0.0,0.036175,0.834337,0.919679,0.117885,0.540,0.488358,0.059419,0.8
002H0J6N0wKjjRtFQhKDzZ,0.295150,0.480430,0.181818,0.711602,0.0,0.037942,0.530120,0.324297,0.334616,0.752,0.584514,0.097124,0.8


In [31]:
df_recommender_scaled.reset_index(inplace=True)
df_recommender.reset_index(inplace=True)

Convert every genre to a feature. If a song is part of a genre it should contain the value 1 otherwise 0.

In [32]:
ct = pd.crosstab(df['spotify_id'], df['genres'])
ct.reset_index(inplace=True)

In [33]:
display(ct.head())
ct.shape

genres,spotify_id,acoustic,afrobeat,alt-rock,alternative,ambient,anime,bluegrass,blues,brazil,...,sleep,songwriter,soul,spanish,swedish,synth-pop,tango,techno,trance,turkish
0,000P83HDtOHcNVFZy7Q2Yu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000paUq6xoYcvCz2REJdu1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0017A9myIXhXYB4gBsPRsA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,001reycpvPkUjmpxTJv6eV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,002H0J6N0wKjjRtFQhKDzZ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


(38456, 96)

Merge both DataFrames together to create the Recommender System.

In [34]:
ct_merged = ct.merge(df_recommender_scaled, on=['spotify_id'], how='left')

In [35]:
ct_merged.set_index(['spotify_id'], inplace=True)

In [36]:
display(ct_merged.head())
ct_merged.shape

,acoustic,afrobeat,alt-rock,alternative,ambient,anime,bluegrass,blues,brazil,british,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,0,0,0,0,0,0,0,0,0,0,...,0.798337,0.0,0.076507,0.439758,0.000042,0.114847,0.696,0.387169,0.067168,0.8
000paUq6xoYcvCz2REJdu1,0,0,0,0,0,0,0,0,0,0,...,0.764918,1.0,0.090229,0.743976,0.037851,0.212072,0.606,0.418638,0.104717,0.8
0017A9myIXhXYB4gBsPRsA,0,0,0,0,0,0,0,0,0,0,...,0.901301,1.0,0.030042,0.084336,0.000004,0.105732,0.360,0.354490,0.069198,0.8
001reycpvPkUjmpxTJv6eV,0,0,0,0,0,0,0,0,0,0,...,0.662231,0.0,0.036175,0.834337,0.919679,0.117885,0.540,0.488358,0.059419,0.8
002H0J6N0wKjjRtFQhKDzZ,0,0,0,0,0,0,0,0,0,0,...,0.711602,0.0,0.037942,0.530120,0.324297,0.334616,0.752,0.584514,0.097124,0.8


(38456, 108)

Overall there are 108 features used for the Recommendation system.

Try different distance measures / similarity functions.

In [38]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, polynomial_kernel, sigmoid_kernel

Compute the cosine similarity between each song. A value close to one means that 2 two songs are equal a value close to 0 means, that they are completley different.

Cosine similarity

In [37]:
cosine_similarity_array = cosine_similarity(ct_merged)

In [39]:
cosine_similarity_array.shape

(38456, 38456)

In [40]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=ct_merged.index, columns=ct_merged.index)

In [41]:
cosine_similarity_df

spotify_id,000P83HDtOHcNVFZy7Q2Yu,000paUq6xoYcvCz2REJdu1,0017A9myIXhXYB4gBsPRsA,001reycpvPkUjmpxTJv6eV,002H0J6N0wKjjRtFQhKDzZ,002Ra2MFY7R5ci6q1YLWLH,003787JNriexFRPKRwyMTn,003FCWl5ci9MxCaiTbErSr,004223m8Zb9ZHcxHFxlpQc,006XjVYo0HbGrUfvL4FodP,...,7zn1gpif2DHi9t9isA5Boe,7znOzcWy38z63lIcKOQBd4,7zndxriU3fd1TkV9BMdv5Y,7zraaKkjSygFzRAObStH8C,7zrlCQpCrfAxWEop9J8OzK,7zsH8mPq4nPBqqqDJpjgJe,7zv3sIP0Umtz9MswoJSF1T,7zwHfIlTAIPa6hgjzK1MGx,7zwlOlX3MA7BFkmf0c1y4p,7zzdiBe3usdBtyqHEekCEs
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,1.000000,0.584950,0.547657,0.577832,0.615026,0.733066,0.436763,0.696065,0.604715,0.440115,...,0.653633,0.571652,0.590742,0.704053,0.510880,0.656446,0.619765,0.633239,0.511864,0.490575
000paUq6xoYcvCz2REJdu1,0.584950,1.000000,0.850264,0.483625,0.521155,0.757498,0.552569,0.725537,0.476348,0.562444,...,0.712973,0.616512,0.877059,0.708038,0.574449,0.723518,0.538335,0.676929,0.567409,0.531816
0017A9myIXhXYB4gBsPRsA,0.547657,0.850264,1.000000,0.398664,0.490335,0.702231,0.499048,0.689698,0.500751,0.492929,...,0.733256,0.654258,0.722472,0.733983,0.611845,0.790129,0.437428,0.627054,0.495387,0.591632
001reycpvPkUjmpxTJv6eV,0.577832,0.483625,0.398664,1.000000,0.554062,0.586211,0.539577,0.549387,0.429293,0.606362,...,0.529015,0.422936,0.467465,0.509193,0.377041,0.491403,0.569164,0.656092,0.570760,0.369105
002H0J6N0wKjjRtFQhKDzZ,0.615026,0.521155,0.490335,0.554062,1.000000,0.576255,0.427358,0.589067,0.504267,0.378358,...,0.576012,0.461075,0.497093,0.582196,0.451617,0.551722,0.543438,0.599098,0.459383,0.575584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7zsH8mPq4nPBqqqDJpjgJe,0.656446,0.723518,0.790129,0.491403,0.551722,0.853721,0.366343,0.582482,0.536233,0.379025,...,0.557212,0.524365,0.682702,0.606600,0.443794,1.000000,0.536788,0.524020,0.413029,0.707870
7zv3sIP0Umtz9MswoJSF1T,0.619765,0.538335,0.437428,0.569164,0.543438,0.659313,0.482791,0.584925,0.439193,0.520979,...,0.518325,0.690803,0.512768,0.522064,0.594918,0.536788,1.000000,0.551991,0.509148,0.411959
7zwHfIlTAIPa6hgjzK1MGx,0.633239,0.676929,0.627054,0.656092,0.599098,0.614871,0.688482,0.765184,0.495488,0.571192,...,0.758185,0.627979,0.635396,0.736928,0.587861,0.524020,0.551991,1.000000,0.723465,0.384070


Linear kernel

In [42]:
linear_kernel_array = linear_kernel(ct_merged)

In [43]:
linear_kernel_array.shape

(38456, 38456)

In [44]:
linear_kernel_df = pd.DataFrame(linear_kernel_array, index=ct_merged.index, columns=ct_merged.index)
linear_kernel_df.head()

spotify_id,000P83HDtOHcNVFZy7Q2Yu,000paUq6xoYcvCz2REJdu1,0017A9myIXhXYB4gBsPRsA,001reycpvPkUjmpxTJv6eV,002H0J6N0wKjjRtFQhKDzZ,002Ra2MFY7R5ci6q1YLWLH,003787JNriexFRPKRwyMTn,003FCWl5ci9MxCaiTbErSr,004223m8Zb9ZHcxHFxlpQc,006XjVYo0HbGrUfvL4FodP,...,7zn1gpif2DHi9t9isA5Boe,7znOzcWy38z63lIcKOQBd4,7zndxriU3fd1TkV9BMdv5Y,7zraaKkjSygFzRAObStH8C,7zrlCQpCrfAxWEop9J8OzK,7zsH8mPq4nPBqqqDJpjgJe,7zv3sIP0Umtz9MswoJSF1T,7zwHfIlTAIPa6hgjzK1MGx,7zwlOlX3MA7BFkmf0c1y4p,7zzdiBe3usdBtyqHEekCEs
spotify_id,,,,,,,,,,,,,,,,,,,,,
000P83HDtOHcNVFZy7Q2Yu,4.256930,2.953650,2.324010,2.831302,2.812805,3.180754,2.155379,3.570523,3.197774,2.039307,...,3.181161,2.756306,3.610660,3.685095,2.730652,2.445887,2.338063,3.375093,2.689898,2.574915
000paUq6xoYcvCz2REJdu1,2.953650,5.989404,4.279828,2.810841,2.827201,3.898633,3.234504,4.414538,2.987895,3.091287,...,4.115933,3.525990,6.358596,4.395858,3.642021,3.197646,2.408937,4.279614,3.536887,3.311023
0017A9myIXhXYB4gBsPRsA,2.324010,4.279828,4.230210,1.947260,2.235485,3.037384,2.455009,3.526743,2.639685,2.276847,...,3.557461,3.144690,4.401928,3.829674,3.260027,2.934732,1.645009,3.331622,2.595126,3.095574
001reycpvPkUjmpxTJv6eV,2.831302,2.810841,1.947260,5.639912,2.916704,2.927719,3.064924,3.243758,2.612997,3.233974,...,2.963520,2.347248,3.288713,3.067704,2.319659,2.107479,2.471463,4.025043,3.452413,2.229948
002H0J6N0wKjjRtFQhKDzZ,2.812805,2.827201,2.235485,2.916704,4.913558,2.686284,2.265792,3.246359,2.864889,1.883516,...,3.011848,2.388455,3.264198,3.273877,2.593389,2.208554,2.202567,3.430563,2.593617,3.245755


In [ ]:
df.sort_values('popularity', ascending=False).head(10)

In [51]:
def recommend_tracks(track: str, df_rec: pd.DataFrame):
    '''
    Recommends tracks that are similar to the provided track.

    Parameter
    ---------
    track: str
        Provided track

    df: pd.DataFrame
        DataFrame used for the Recommendation
    
    '''
    ids = df[df.name == track]['spotify_id'].values
    for id in ids:
        recommender_df = df_rec[[ids[0]]]
        recommender_df.reset_index(inplace=True)
        recommender_df = recommender_df.rename(columns={id: 'Score'})
        recommender_df = recommender_df.merge(df_total, how='left', on='spotify_id')
        display(recommender_df.sort_values(by='Score', ascending=False).head(10))


In [52]:
recommend_tracks('Back In Black', cosine_similarity_df)

,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
722,08mG3Y1vljYA6bvDt4Wqkj,1.000000,"[hard-rock, rock]","[Back In Black, Back In Black]","[AC/DC, AC/DC]","[Back In Black, Back In Black]","[1980-07-25, 1980-07-25]","[day, day]","[spotify:track:08mG3Y1vljYA6bvDt4Wqkj, spotify...","[AUAP08000046, AUAP08000046]",...,-5.678,1.0,0.0470,0.011000,0.009650,0.0828,0.763,188.386,255493.0,4.0
25422,5AD7oNm0kvWIXxW3vy0z2a,0.996716,"[hard-rock, rock]","[Homemade Love, Homemade Love]","[Journey, Journey]","[Departure, Departure]","[1980, 1980]","[year, year]","[spotify:track:5AD7oNm0kvWIXxW3vy0z2a, spotify...","[USSM18000101, USSM18000101]",...,-7.733,1.0,0.0935,0.024400,0.000009,0.1690,0.758,178.672,172560.0,4.0
26519,5O4rVUjsG66l2nArhYW57m,0.996298,"[hard-rock, rock]","[Relentless, Relentless]","[Kansas, Kansas]","[Audio-Visions, Audio-Visions]","[1980-09-01, 1980-09-01]","[day, day]","[spotify:track:5O4rVUjsG66l2nArhYW57m, spotify...","[USSM11102555, USSM11102555]",...,-6.793,1.0,0.0338,0.000872,0.000003,0.0543,0.749,139.839,296813.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,0.992350,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
18529,3m3hmciMkd4y3XGC1mjTHf,0.991354,"[hard-rock, rock]","[Rolê da Estrada, Rolê da Estrada]","[Patrulha do Espaço, Patrulha do Espaço]","[Patrulha do Espaço, Patrulha do Espaço]","[1980-10-10, 1980-10-10]","[day, day]","[spotify:track:3m3hmciMkd4y3XGC1mjTHf, spotify...","[BR3QI1300004, BR3QI1300004]",...,-4.075,1.0,0.0358,0.032100,0.010300,0.1040,0.742,148.813,142000.0,4.0
36341,7MxmJmW08fXXIx0WV1UeKz,0.990243,"[hard-rock, rock]","[Someone Tonight, Someone Tonight]","[REO Speedwagon, REO Speedwagon]","[Hi Infidelity (30th Anniversary Edition), Hi ...","[1980, 1980]","[year, year]","[spotify:track:7MxmJmW08fXXIx0WV1UeKz, spotify...","[USSM11102624, USSM11102624]",...,-5.425,1.0,0.0437,0.075100,0.000000,0.1330,0.855,133.617,160240.0,4.0
25670,5DVXOyNMl4MunWZ2BDvgA5,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night (Remastered), On Through...","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:5DVXOyNMl4MunWZ2BDvgA5, spotify...","[GBUM71802087, GBUM71802087]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
25215,57hFsLrkALFFIxncLAY7Tg,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night, On Through The Night]","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:57hFsLrkALFFIxncLAY7Tg, spotify...","[GBF087900785, GBF087900785]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
16911,3R306PKKOPAUjSSMkDHOpY,0.988208,"[hard-rock, rock]","[Rock n' Roll Is Alive and Well, Rock n' Roll ...","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:3R306PKKOPAUjSSMkDHOpY, spotify...","[GBAYK8000176, GBAYK8000176]",...,-6.773,1.0,0.0458,0.026500,0.000008,0.2780,0.539,172.996,251600.0,4.0
21489,4NyZk1OfjwkGiIQxsD10ii,0.987438,"[hard-rock, rock]","[Anytime, Anytime]","[The Babys, The Babys]","[Union Jacks, Union Jacks]","[1980-01-01, 1980-01-01]","[day, day]","[spotify:track:4NyZk1OfjwkGiIQxsD10ii, spotify...","[GBAYK0700064, GBAYK0700064]",...,-5.763,1.0,0.0304,0.008980,0.000000,0.1760,0.767,112.699,202760.0,4.0


,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
722,08mG3Y1vljYA6bvDt4Wqkj,1.000000,"[hard-rock, rock]","[Back In Black, Back In Black]","[AC/DC, AC/DC]","[Back In Black, Back In Black]","[1980-07-25, 1980-07-25]","[day, day]","[spotify:track:08mG3Y1vljYA6bvDt4Wqkj, spotify...","[AUAP08000046, AUAP08000046]",...,-5.678,1.0,0.0470,0.011000,0.009650,0.0828,0.763,188.386,255493.0,4.0
25422,5AD7oNm0kvWIXxW3vy0z2a,0.996716,"[hard-rock, rock]","[Homemade Love, Homemade Love]","[Journey, Journey]","[Departure, Departure]","[1980, 1980]","[year, year]","[spotify:track:5AD7oNm0kvWIXxW3vy0z2a, spotify...","[USSM18000101, USSM18000101]",...,-7.733,1.0,0.0935,0.024400,0.000009,0.1690,0.758,178.672,172560.0,4.0
26519,5O4rVUjsG66l2nArhYW57m,0.996298,"[hard-rock, rock]","[Relentless, Relentless]","[Kansas, Kansas]","[Audio-Visions, Audio-Visions]","[1980-09-01, 1980-09-01]","[day, day]","[spotify:track:5O4rVUjsG66l2nArhYW57m, spotify...","[USSM11102555, USSM11102555]",...,-6.793,1.0,0.0338,0.000872,0.000003,0.0543,0.749,139.839,296813.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,0.992350,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
18529,3m3hmciMkd4y3XGC1mjTHf,0.991354,"[hard-rock, rock]","[Rolê da Estrada, Rolê da Estrada]","[Patrulha do Espaço, Patrulha do Espaço]","[Patrulha do Espaço, Patrulha do Espaço]","[1980-10-10, 1980-10-10]","[day, day]","[spotify:track:3m3hmciMkd4y3XGC1mjTHf, spotify...","[BR3QI1300004, BR3QI1300004]",...,-4.075,1.0,0.0358,0.032100,0.010300,0.1040,0.742,148.813,142000.0,4.0
36341,7MxmJmW08fXXIx0WV1UeKz,0.990243,"[hard-rock, rock]","[Someone Tonight, Someone Tonight]","[REO Speedwagon, REO Speedwagon]","[Hi Infidelity (30th Anniversary Edition), Hi ...","[1980, 1980]","[year, year]","[spotify:track:7MxmJmW08fXXIx0WV1UeKz, spotify...","[USSM11102624, USSM11102624]",...,-5.425,1.0,0.0437,0.075100,0.000000,0.1330,0.855,133.617,160240.0,4.0
25670,5DVXOyNMl4MunWZ2BDvgA5,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night (Remastered), On Through...","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:5DVXOyNMl4MunWZ2BDvgA5, spotify...","[GBUM71802087, GBUM71802087]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
25215,57hFsLrkALFFIxncLAY7Tg,0.989644,"[hard-rock, rock]","[Hello America, Hello America]","[Def Leppard, Def Leppard]","[On Through The Night, On Through The Night]","[1980-03-14, 1980-03-14]","[day, day]","[spotify:track:57hFsLrkALFFIxncLAY7Tg, spotify...","[GBF087900785, GBF087900785]",...,-6.776,1.0,0.0827,0.001990,0.195000,0.1140,0.635,155.783,207520.0,4.0
16911,3R306PKKOPAUjSSMkDHOpY,0.988208,"[hard-rock, rock]","[Rock n' Roll Is Alive and Well, Rock n' Roll ...","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:3R306PKKOPAUjSSMkDHOpY, spotify...","[GBAYK8000176, GBAYK8000176]",...,-6.773,1.0,0.0458,0.026500,0.000008,0.2780,0.539,172.996,251600.0,4.0
21489,4NyZk1OfjwkGiIQxsD10ii,0.987438,"[hard-rock, rock]","[Anytime, Anytime]","[The Babys, The Babys]","[Union Jacks, Union Jacks]","[1980-01-01, 1980-01-01]","[day, day]","[spotify:track:4NyZk1OfjwkGiIQxsD10ii, spotify...","[GBAYK0700064, GBAYK0700064]",...,-5.763,1.0,0.0304,0.008980,0.000000,0.1760,0.767,112.699,202760.0,4.0


In [53]:
recommend_tracks('Back In Black', linear_kernel_df)

,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
2112,0PLzoe4izqluPTVsF9K4mb,7.155934,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0PLzoe4izqluPTVsF9K4mb, spotify...","[FILKA8000375, FILKA8000375, FILKA8000375, FIL...",...,-4.752,1.0,0.0726,0.005900,0.000000,0.3390,0.745,173.004,165680.0,4.0
4650,0vEEhp1BFWt1iCKsojoebd,7.150820,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ska 4, Ska 4, Ska 4, Ska 4, Ska 4]","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0vEEhp1BFWt1iCKsojoebd, spotify...","[FILKA8000369, FILKA8000369, FILKA8000369, FIL...",...,-4.207,1.0,0.2660,0.005240,0.016700,0.1700,0.812,170.904,141800.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,7.114456,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
1578,0Ix7jl28ovNUGclUmkwHnx,7.092672,"[hard-rock, rock]","[My Life, My Life]","[Peter Criss, Peter Criss]","[Out Of Control, Out Of Control]","[1980, 1980]","[year, year]","[spotify:track:0Ix7jl28ovNUGclUmkwHnx, spotify...","[USMR18070057, USMR18070057]",...,-5.082,1.0,0.0339,0.147000,0.000007,0.0634,0.877,119.227,222933.0,4.0
28209,5joQhLd0l0jc1S0U9TkmsG,7.073923,"[alt-rock, alternative, blues, hard-rock, hard...","[Soul Kitchen, Soul Kitchen, Soul Kitchen, Sou...","[X, X, X, X, X, X, X, X, X]","[Los Angeles, Los Angeles, Los Angeles, Los An...","[1980-04-26, 1980-04-26, 1980-04-26, 1980-04-2...","[day, day, day, day, day, day, day, day, day]","[spotify:track:5joQhLd0l0jc1S0U9TkmsG, spotify...","[USFP71869503, USFP71869503, USFP71869503, USF...",...,-4.304,1.0,0.0442,0.002030,0.001110,0.3630,0.836,101.277,145467.0,4.0
29747,63YhDDyYQJo847NqN7mry3,7.066416,"[alt-rock, alternative, blues, grunge, hard-ro...","[Cover Band - Demo, Cover Band - Demo, Cover B...","[Redd Kross, Redd Kross, Redd Kross, Redd Kros...","[Red Cross, Red Cross, Red Cross, Red Cross, R...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day, day, day, day, day, ...","[spotify:track:63YhDDyYQJo847NqN7mry3, spotify...","[USMRG2073408, USMRG2073408, USMRG2073408, USM...",...,-5.967,1.0,0.0743,0.000029,0.002410,0.1480,0.719,182.770,89960.0,4.0
17610,3Zw7u9z5sfbasLy9zkubzn,7.045456,"[blues, british, hard-rock, metal, rock]","[Carry Your Load, Carry Your Load, Carry Your ...","[Whitesnake, Whitesnake, Whitesnake, Whitesnak...","[Ready An' Willing, Ready An' Willing, Ready A...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day]","[spotify:track:3Zw7u9z5sfbasLy9zkubzn, spotify...","[CAU111120901, CAU111120901, CAU111120901, CAU...",...,-4.309,1.0,0.0312,0.048200,0.000004,0.0333,0.897,150.405,244520.0,4.0
30396,6BUuoR1kG1rRag718ibuOn,7.043061,"[country, folk, hard-rock, rock]","[Betty Lou's Gettin' Out Tonight, Betty Lou's ...","[Bob Seger, Bob Seger, Bob Seger, Bob Seger]","[Against The Wind, Against The Wind, Against T...","[1980-02-25, 1980-02-25, 1980-02-25, 1980-02-25]","[day, day, day, day]","[spotify:track:6BUuoR1kG1rRag718ibuOn, spotify...","[USCA28500277, USCA28500277, USCA28500277, USC...",...,-3.081,1.0,0.0506,0.142000,0.126000,0.2930,0.972,170.199,171920.0,4.0
12883,2cXADTZcYNhvtfZuK77Xt0,7.040993,"[blues, hard-rock, metal, rock]","[Hungry Boys, Hungry Boys, Hungry Boys, Hungry...","[Blue Öyster Cult, Blue Öy

,spotify_id,Score,genres,name,artists,album,release_date,release_date_precision,uri,isrc,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
2112,0PLzoe4izqluPTVsF9K4mb,7.155934,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0PLzoe4izqluPTVsF9K4mb, spotify...","[FILKA8000375, FILKA8000375, FILKA8000375, FIL...",...,-4.752,1.0,0.0726,0.005900,0.000000,0.3390,0.745,173.004,165680.0,4.0
4650,0vEEhp1BFWt1iCKsojoebd,7.150820,"[hard-rock, hardcore, punk, punk-rock, rock]","[Ska 4, Ska 4, Ska 4, Ska 4, Ska 4]","[Maukka Perusjätkä,Ralf Örn Pop, Maukka Perusj...","[Ennen kolmatta maailmansotaa, Ennen kolmatta ...","[1980-12-12, 1980-12-12, 1980-12-12, 1980-12-1...","[day, day, day, day, day]","[spotify:track:0vEEhp1BFWt1iCKsojoebd, spotify...","[FILKA8000369, FILKA8000369, FILKA8000369, FIL...",...,-4.207,1.0,0.2660,0.005240,0.016700,0.1700,0.812,170.904,141800.0,4.0
6199,1En7X8Ama1U8MosJbDndnB,7.114456,"[hard-rock, rock]","[Too Far Gone, Too Far Gone]","[The Babys, The Babys]","[On the Edge, On the Edge]","[1980-10-01, 1980-10-01]","[day, day]","[spotify:track:1En7X8Ama1U8MosJbDndnB, spotify...","[GBAYK0700073, GBAYK0700073]",...,-6.881,1.0,0.0602,0.019700,0.093000,0.1910,0.930,169.456,175000.0,4.0
1578,0Ix7jl28ovNUGclUmkwHnx,7.092672,"[hard-rock, rock]","[My Life, My Life]","[Peter Criss, Peter Criss]","[Out Of Control, Out Of Control]","[1980, 1980]","[year, year]","[spotify:track:0Ix7jl28ovNUGclUmkwHnx, spotify...","[USMR18070057, USMR18070057]",...,-5.082,1.0,0.0339,0.147000,0.000007,0.0634,0.877,119.227,222933.0,4.0
28209,5joQhLd0l0jc1S0U9TkmsG,7.073923,"[alt-rock, alternative, blues, hard-rock, hard...","[Soul Kitchen, Soul Kitchen, Soul Kitchen, Sou...","[X, X, X, X, X, X, X, X, X]","[Los Angeles, Los Angeles, Los Angeles, Los An...","[1980-04-26, 1980-04-26, 1980-04-26, 1980-04-2...","[day, day, day, day, day, day, day, day, day]","[spotify:track:5joQhLd0l0jc1S0U9TkmsG, spotify...","[USFP71869503, USFP71869503, USFP71869503, USF...",...,-4.304,1.0,0.0442,0.002030,0.001110,0.3630,0.836,101.277,145467.0,4.0
29747,63YhDDyYQJo847NqN7mry3,7.066416,"[alt-rock, alternative, blues, grunge, hard-ro...","[Cover Band - Demo, Cover Band - Demo, Cover B...","[Redd Kross, Redd Kross, Redd Kross, Redd Kros...","[Red Cross, Red Cross, Red Cross, Red Cross, R...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day, day, day, day, day, ...","[spotify:track:63YhDDyYQJo847NqN7mry3, spotify...","[USMRG2073408, USMRG2073408, USMRG2073408, USM...",...,-5.967,1.0,0.0743,0.000029,0.002410,0.1480,0.719,182.770,89960.0,4.0
17610,3Zw7u9z5sfbasLy9zkubzn,7.045456,"[blues, british, hard-rock, metal, rock]","[Carry Your Load, Carry Your Load, Carry Your ...","[Whitesnake, Whitesnake, Whitesnake, Whitesnak...","[Ready An' Willing, Ready An' Willing, Ready A...","[1980-01-01, 1980-01-01, 1980-01-01, 1980-01-0...","[day, day, day, day, day]","[spotify:track:3Zw7u9z5sfbasLy9zkubzn, spotify...","[CAU111120901, CAU111120901, CAU111120901, CAU...",...,-4.309,1.0,0.0312,0.048200,0.000004,0.0333,0.897,150.405,244520.0,4.0
30396,6BUuoR1kG1rRag718ibuOn,7.043061,"[country, folk, hard-rock, rock]","[Betty Lou's Gettin' Out Tonight, Betty Lou's ...","[Bob Seger, Bob Seger, Bob Seger, Bob Seger]","[Against The Wind, Against The Wind, Against T...","[1980-02-25, 1980-02-25, 1980-02-25, 1980-02-25]","[day, day, day, day]","[spotify:track:6BUuoR1kG1rRag718ibuOn, spotify...","[USCA28500277, USCA28500277, USCA28500277, USC...",...,-3.081,1.0,0.0506,0.142000,0.126000,0.2930,0.972,170.199,171920.0,4.0
12883,2cXADTZcYNhvtfZuK77Xt0,7.040993,"[blues, hard-rock, metal, rock]","[Hungry Boys, Hungry Boys, Hungry Boys, Hungry...","[Blue Öyster Cult, Blue Öy

In [54]:
import pandas as pd

In [55]:
colums = ['Vorname', 'Nachname']

df = pd.DataFrame([['Jannis', 'Weichler'], ['Lite', 'Brandt']], columns=colums)

In [61]:
df.to_csv('names.csv', index=False)
df.to_csv('names.csv', index=False)

In [62]:
pd.read_csv('names.csv')

,Vorname,Nachname
0,Jannis,Weichler
1,Lite,Brandt
